In [ ]:
import os
from io import BytesIO
from google import genai
from google.genai.errors import APIError
from PIL import Image

# ----------------------------------------------------
# 1. 설정 및 상수
# ----------------------------------------------------

# AI 모델이 안정적으로 처리할 수 있는 최대 입력 해상도 (Gemini/Imagen 권장 규격)
MAX_DIMENSION = 1024 
OUTPUT_FORMAT = 'JPEG'
OUTPUT_QUALITY = 90

# ----------------------------------------------------
# 2. 전처리: 리사이징 및 비율 관리
# ----------------------------------------------------

def preprocess_image(image_path: str, max_dim: int = MAX_DIMENSION) -> tuple[Image.Image, float, tuple[int, int]]:
    """
    이미지를 로드하고, 비율을 유지하며 최대 크기를 초과하지 않게 리사이징하여 전처리합니다.
    """
    original_image = Image.open(image_path).convert('RGB')
    original_width, original_height = original_image.size
    
    # 픽셀 초과 시 리사이징 비율 계산 (원본 비율 유지)
    if max(original_width, original_height) > max_dim:
        scaling_ratio = max_dim / max(original_width, original_height)
        new_width = int(original_width * scaling_ratio)
        new_height = int(original_height * scaling_ratio)
        resized_image = original_image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        print(f"✅ 리사이징 완료: {original_width}x{original_height} -> {new_width}x{new_height}")
    else:
        resized_image = original_image
        scaling_ratio = 1.0
        print(f"✅ 원본 크기 ({original_width}x{original_height})는 규격 내입니다.")
    
    return resized_image, scaling_ratio, (original_width, original_height)

# ----------------------------------------------------
# 3. AI 분석 및 디자인 지침 생성 (Gemini API)
# ----------------------------------------------------

def get_design_instructions(
    client: genai.Client,
    image: Image.Image,
    style_prompt: str
) -> str:
    """
    Gemini 모델을 통해 이미지를 분석하고 Imagen에 전달할 구체적인 지침을 생성합니다.
    """
    # Gemini에게 텍스트 내용의 정확한 인식과 스타일 변경 지침을 요청합니다.
    full_prompt = (
        "제공된 이미지에서 텍스트 내용을 한 글자도 틀리지 않고 정확히 인식하고, "
        "그 레이아웃(위치와 줄바꿈)을 유지하십시오. "
        "원본 배경의 내용은 보존하면서 텍스트 디자인만 다음 스타일로 변경하여 "
        "최종 이미지를 생성하기 위한 완벽한 프롬프트 지침을 **텍스트로만** 작성해 주세요. "
        "새로운 디자인 스타일: "
        f"'{style_prompt}'"
    )
    
    print("🧠 Gemini 분석 시작: 텍스트 인식 및 디자인 지침 생성 중...")
    
    response = client.models.generate_content(
        model='gemini-2.5-pro',
        contents=[full_prompt, image],
        config={"max_output_tokens": 4096}
    )
    
    return response.text

# ----------------------------------------------------
# 4. 이미지 재합성 (Imagen API - 논리적 구현)
# ----------------------------------------------------

def generate_and_recombine_image(design_instructions: str, resized_image: Image.Image) -> Image.Image:
    """
    (가상 함수): Imagen API를 호출하여 이미지를 재합성하고 결과를 반환합니다.
    
    실제 구현 시: Vertex AI SDK를 사용하여 Imagen 모델의 Inpainting/Image-to-Image 기능을 호출합니다.
    """
    print("\n💡 Imagen API 호출 단계 (실제 구현 시 Vertex AI SDK 필요)")
    print(f"   * 최종 디자인 프롬프트 (Imagen으로 전달): {design_instructions[:100]}...")
    
    # **********************************************************
    # * 여기에 실제 Imagen API 호출 로직이 들어갑니다.           *
    # * *
    # * 1. Imagen 모델 로드 및 인증                            *
    # * 2. 프롬프트와 원본 이미지(resized_image)를 인페인팅 모드로 전달*
    # * 3. AI가 처리한 새로운 이미지 객체를 반환                *
    # **********************************************************
    
    # **현재는 실제 API 호출이 불가능하므로, 임시로 원본 이미지를 그대로 반환합니다.**
    return resized_image 

# ----------------------------------------------------
# 5. 후처리: 원본 해상도로 복원 및 저장
# ----------------------------------------------------

def postprocess_image(processed_image: Image.Image, original_size: tuple[int, int], output_path: str):
    """
    처리된 이미지를 원본 해상도로 업스케일링하고 저장합니다.
    """
    # 1. 업스케일링 (원본 해상도로 복원)
    final_image = processed_image.resize(original_size, Image.Resampling.LANCZOS)
    
    # 2. 저장
    final_image.save(output_path, format=OUTPUT_FORMAT, quality=OUTPUT_QUALITY)
    print(f"\n✅ 최종 이미지 저장 완료: {output_path} (원본 해상도 {original_size[0]}x{original_size[1]})")

# ----------------------------------------------------
# 6. 메인 실행 함수
# ----------------------------------------------------

def run(image_path: str):
    """
    전체 이미지 처리 및 디자인 로직을 실행하는 메인 함수입니다.
    """
    # --- 사용자 설정 ---
    OUTPUT_FILE = 'styled_banner_final_output.jpg'
    CUSTOM_STYLE = (
        "배경의 화려한 전통적인 색상과 조화되도록 글씨를 **채도 높은 금색 바탕에 두꺼운 검정색 테두리**를 사용하고, "
        "글씨체는 **한국적인 느낌이 나는 굵은 명조체**로 변경하여 축제 포스터의 중앙에 선명하게 인쇄된 듯한 질감을 주세요."
    )
    
    try:
        # 0. API 클라이언트 초기화
        if 'GEMINI_API_KEY' not in os.environ:
            raise EnvironmentError(
                "GEMINI_API_KEY 환경 변수가 설정되지 않았습니다."
            )
        client = genai.Client()

        print(f"\n--- 🚀 이미지 디자인 통합 프로세스 시작: {image_path} ---")
        
        # 1. 전처리: 리사이징 및 비율 정보 저장
        resized_img, scaling_ratio, original_size = preprocess_image(image_path)
        
        # 2. AI 분석: Gemini를 통해 디자인 지침 획득
        design_instructions = get_design_instructions(client, resized_img, CUSTOM_STYLE)
        
        # 3. 이미지 재합성: Imagen API 호출 (논리적 실행)
        # 이 함수가 실제로는 디자인이 변경된 이미지를 반환해야 합니다.
        processed_img = generate_and_recombine_image(design_instructions, resized_img)

        # 4. 후처리: 원본 해상도로 복원 및 저장
        postprocess_image(processed_img, original_size, OUTPUT_FILE)

        print("\n--- ✅ 프로세스 성공적으로 완료 ---")

    except Exception as e:
        print(f"⛔ 실행 오류 발생: {e}")


if __name__ == '__main__':
    # -------------------------------------------------------------------------
    # 테스트 실행
    # -------------------------------------------------------------------------
    
    # 1. API Key 설정 (실제 키로 대체 필요)
    # os.environ['GEMINI_API_KEY'] = 'YOUR_API_KEY'

    # 2. 테스트에 사용할 이미지 경로 (사용자 파일명 지정)
    # 예시: 가로등 배너_4 (1).jpg
    TEST_IMAGE_PATH = '가로등 배너_4 (1).jpg' 
    
    if os.path.exists(TEST_IMAGE_PATH):
        run(TEST_IMAGE_PATH)
    else:
        print(f"⚠️ 경고: 테스트 파일 '{TEST_IMAGE_PATH}'을 찾을 수 없습니다.")

📥 Input image: test_input.png
✅ Output saved: gpt_image_render_output\gpt_image_render_45cb399df3194e4e804974fc9fc3446f.png


WindowsPath('gpt_image_render_output/gpt_image_render_45cb399df3194e4e804974fc9fc3446f.png')